<a href="https://colab.research.google.com/github/before-born/Neuron/blob/main/ObjectDetectionUsingCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Load the pre-trained object detection model from TensorFlow Hub
model_url = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(model_url).signatures['default']

# Load and preprocess the image
def load_image(image_path):
    img = Image.open(image_path)
    img = img.convert('RGB')  # Ensure image is in RGB format
    img_np = np.array(img)
    return img_np

def preprocess_image(image_np):
    image_tensor = tf.convert_to_tensor(image_np, dtype=tf.float32)
    image_tensor = tf.image.convert_image_dtype(image_tensor, dtype=tf.uint8)
    image_tensor = tf.expand_dims(image_tensor, 0)  # Add batch dimension
    return image_tensor

# Perform object detection on the input image
def detect_objects(image_tensor):
    result = detector(image_tensor)
    return result

def draw_boxes(image_np, boxes, class_ids, scores):
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image_np)

    for i in range(len(boxes)):
        box = boxes[i]
        class_id = class_ids[i]
        score = scores[i]

        # Draw bounding box
        ymin, xmin, ymax, xmax = box
        rect = patches.Rectangle((xmin * image_np.shape[1], ymin * image_np.shape[0]),
                                 (xmax - xmin) * image_np.shape[1],
                                 (ymax - ymin) * image_np.shape[0],
                                 linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(xmin * image_np.shape[1], ymin * image_np.shape[0],
                 f'{class_id} ({score:.2f})', color='red', fontsize=12, bbox=dict(facecolor='yellow', alpha=0.5))

    plt.show()

# Example usage
image_path = 'path_to_your_image.jpg'  # Replace with the path to your image
image_np = load_image(image_path)
image_tensor = preprocess_image(image_np)
result = detect_objects(image_tensor)

# Extract and draw results
boxes = result['detection_boxes'].numpy()[0]
class_ids = result['detection_classes'].numpy()[0]
scores = result['detection_scores'].numpy()[0]

draw_boxes(image_np, boxes, class_ids, scores)
